# Active Learning for Sparse Semantic Segmentation

This notebook provides a simple workflow for training semantic segmentation models using sparse point annotations through active learning.

## Workflow

**One-Time Setup:**
1. **Cell 1**: Setup and Imports
2. **Cell 2**: Configure Dataset + Model
3. **Cell 3**: Create/Load Session

**Active Learning Loop:**
4. **Cell 4**: Annotate (launch tool, add/edit points)
5. **Cell 5**: Train (train model, generate predictions, create next iteration)
6. **Repeat**: Cell 4 → Cell 5 → Cell 4 → Cell 5...

## Requirements

- PyTorch 1.10+
- segmentation_models_pytorch
- PyQt5 (for annotation tool)
- See `requirements.txt` for full dependencies

---
## Cell 1: Setup and Imports

Import required libraries and set up the environment.

In [1]:
import sys
from pathlib import Path
import json
import shutil
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import imageio
import pandas as pd
from tqdm import tqdm

# Add project root to path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import project modules
from src.utils.config_loader import load_dataset_config, load_training_config
from src.session.mask_utils import batch_json_to_masks
from src.session.simple_mask_converter import convert_mask_to_json

# Check PyTorch
try:
    import torch
    import torch.utils.data as data
    from torchvision import transforms
    print(f"✓ PyTorch {torch.__version__} available")
    print(f"✓ CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"✓ CUDA device: {torch.cuda.get_device_name(0)}")
except ImportError:
    print("✗ PyTorch not found. Install with: conda install pytorch torchvision cudatoolkit -c pytorch")
    raise

# Check segmentation_models_pytorch
try:
    import segmentation_models_pytorch as smp
    print(f"✓ segmentation_models_pytorch available")
except ImportError:
    print("✗ segmentation_models_pytorch not found. Install with: pip install segmentation-models-pytorch")
    raise

# Set matplotlib style
plt.style.use('default')
%matplotlib inline

print("\n✓ All imports successful")
print(f"✓ Project root: {project_root}")

✓ PyTorch 2.6.0+cu126 available
✓ CUDA available: True
✓ CUDA device: NVIDIA GeForce RTX 4090
✓ segmentation_models_pytorch available

✓ All imports successful
✓ Project root: d:\SIAL


---
## Cell 2: Configure Dataset + Model

Load configurations and create model, loss, optimizer, and metrics.

In [2]:
# Configuration paths
DATASET_CONFIG_PATH = 'configs/datasets/vaihingen_1k_v3.yaml'
TRAINING_CONFIG_PATH = 'configs/training/unet_efficientnet_b7.yaml'

# Load configurations
print("Loading configurations...\n")

dataset_config = load_dataset_config(DATASET_CONFIG_PATH)
print(f"Dataset: {dataset_config['name']}")
print(f"  Classes: {dataset_config['classes']['num_classes']}")
print(f"  Class names: {', '.join(dataset_config['classes']['names'])}")
print(f"  Image size: {dataset_config['image']['width']}x{dataset_config['image']['height']}")
print(f"  Ignore index: {dataset_config['classes']['ignore_index']}")

training_config = load_training_config(TRAINING_CONFIG_PATH)
print(f"\nTraining: {training_config['name']}")
print(f"  Model: {training_config['model']['architecture']}")
print(f"  Encoder: {training_config['model']['encoder']}")
print(f"  Epochs: {training_config['training']['num_epochs']}")
print(f"  Batch size: train={training_config['training']['batch_size']['train']}, val={training_config['training']['batch_size']['val']}")
print(f"  Device: {training_config['training']['device']}")
print(f"  Learning rate: {training_config['optimizer']['params']['lr']}")

# Setup training (model, losses, metrics, optimizer)
print()
from src.training.setup import setup_training

model, device, train_loss, val_loss, metrics, optimizer = setup_training(
    dataset_config=dataset_config,
    training_config=training_config
)

Loading configurations...

Dataset: VAIHINGEN_1K_V3
  Classes: 6
  Class names: impervious, building, tree, car, low_veg, clutter
  Image size: 512x512
  Ignore index: 6

Training: UNet-EfficientNetB7
  Model: Unet
  Encoder: efficientnet-b7
  Epochs: 500
  Batch size: train=10, val=50
  Device: cuda
  Learning rate: 0.0001

Setting up training components...
✓ Model: Unet + efficientnet-b7
✓ Device: cuda
✓ Train Loss: DWCDL (confident_wrong=10.0x, uncertain_wrong=4.0x, uncertain_correct=2.0x)
✓ Val Loss: CrossEntropyLoss
✓ Metrics: mIoU
✓ Optimizer: Adam (lr=0.0001)
✓ Training setup complete



---
## Cell 3: Create/Load Session

Create a new session or load an existing one.

**Session Structure:**
```
Sessions/
  └── {SESSION_NAME}/
      ├── iteration_0/
      │   ├── annotations/  # JSON files
      │   ├── masks/        # PNG masks
      │   ├── models/       # Model weights
      │   └── predictions/  # Predictions
      ├── iteration_1/
      │   └── ...
      └── metrics_history.csv
```

In [3]:
# Session configuration
SESSION_NAME = 'VAIHINGEN_1k'
SESSION_PATH = Path(f'Sessions/{SESSION_NAME}')
LAUNCH_TOOL = True

print(f"Session: {SESSION_NAME}")
print(f"Path: {SESSION_PATH}\n")

# Get or create session (modularized!)
from src.session.manager import get_or_create_session

session_info = get_or_create_session(
    session_path=SESSION_PATH,
    dataset_config=dataset_config
)

print(f"\n{'='*60}")
print(f"SESSION READY")
print(f"{'='*60}")
print(f"→ Run Cell 4 to annotate")
print(f"→ Run Cell 5 to train")

Session: VAIHINGEN_1k
Path: Sessions\VAIHINGEN_1k

Loading existing session...

Available iterations: [0, 1, 2, 3, 4]
Latest iteration: 4

Iteration 4 status:
  Annotations: ✓ (1000 files)
  Masks:       ✓ (1000 files)
  Model:       ✗
  Predictions: ✗ (0 files)

Metrics history:
 iteration     miou  pixel_accuracy  train_loss  val_loss
         0 0.501606        0.792913    0.469387  1.258772
         1 0.562324        0.836759    0.555166  1.217716
         2 0.584286        0.844113    0.590927  1.198189
         3 0.602103        0.855051    0.605222  1.195125

✓ Session loaded: Sessions\VAIHINGEN_1k

SESSION READY
→ Run Cell 4 to annotate
→ Run Cell 5 to train


---
## Cell 4: Annotate 🎨

Launch the annotation tool to add or refine point annotations.

**What happens:**
1. Finds latest iteration
2. Launches VIZ_SOFTWARE annotation tool (PyQt5 window)
3. Shows images with current annotations
4. Shows predictions from previous iteration (if available)
5. You add/edit/remove annotation points
6. Tool auto-saves to JSONs
7. When you close the tool:
   - Converts JSONs → PNG masks
   - Ready for training

**After this cell:**
- Run Cell 5 to train the model

In [4]:
# Annotation workflow configuration
LAUNCH_TOOL = True  # Set to False to skip annotation tool

# Run annotation workflow (modularized!)
from src.annotation.launcher import run_annotation_workflow

result = run_annotation_workflow(
    session_path=SESSION_PATH,
    dataset_config=dataset_config,
    iteration='latest',  # or 'current', or specific iteration number (0, 1, 2, ...)
    launch_tool=LAUNCH_TOOL
)

ANNOTATION WORKFLOW

Iteration: 4
Session: Sessions\VAIHINGEN_1k

Current status:
  Annotations: 1000 files
  Masks:       1000 files
  Predictions: 1000 files (from iteration 3)

✓ Predictions available - will guide your annotations

------------------------------------------------------------
LAUNCHING ANNOTATION TOOL...
------------------------------------------------------------


✓ ANNOTATION TOOL CLOSED

Converting annotations to masks...

✓ Converted 1000 annotations to masks

ANNOTATION COMPLETE
→ Run Cell 5 to train model


---
## Cell 5: Train 🚀

Train the model on annotated data.

**What happens:**
1. Finds latest iteration
2. Loads masks from `iteration_N/masks/`
3. Trains model (simple training using smp)
4. Saves best model to `iteration_N/models/best_model.pth`
5. Generates predictions → `iteration_N/predictions/`
6. Creates next iteration (`iteration_{N+1}`)
7. Copies annotations to next iteration

**After this cell:**
- Go back to Cell 4 to annotate the next iteration
- The loop continues: Cell 4 → Cell 5 → Cell 4 → Cell 5...

In [ ]:
# Training configuration
VISUALIZE = False  # Set to False to skip visualization plots

# Run training iteration (modularized!)
from src.training.workflow import run_training_iteration

result = run_training_iteration(
    session_path=SESSION_PATH,
    dataset_config=dataset_config,
    training_config=training_config,
    model=model,
    device=device,
    train_loss=train_loss,
    val_loss=val_loss,
    metrics=metrics,
    optimizer=optimizer,
    iteration='latest',  # or 'current', or specific iteration number (0, 1, 2, ...)
    visualize=VISUALIZE
)